# Vector2d

In [89]:
from typing import SupportsComplex
from array import array
import math
from __future__ import annotations  # Postponed Evaluation of Annotations（后续版本成为默认可以不用导入）

class Vector2d:
    __match_args__ = ('x', 'y')
    __slots__ = ('__x', '__y')

    typecode = 'd'

    def __init__(self, x, y) -> None:
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __hash__(self) -> int:
        return hash((self.x, self.y))

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self) -> str:
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, __value) -> bool:
        return tuple(self) == tuple(__value)

    def __abs__(self) -> float:
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec='') -> str:
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

    def __complex__(self) -> complex:
        return complex(self.x, self.y)

    @classmethod
    def fromcomplex(cls, datum: SupportsComplex) -> Vector2d:
        c = complex(datum)
        return cls(c.real, c.imag)

AttributeError: type object 'Vector2d' has no attribute '__annotations__'

In [34]:
from typing import SupportsAbs, SupportsComplex

v = Vector2d(3, 4)
isinstance(v, SupportsAbs), isinstance(v, SupportsComplex)
abs(v), complex(v)

(5.0, (3+4j))

In [35]:
Vector2d.fromcomplex(3+4j)

Vector2d(3.0, 4.0)

In [24]:
class ShortVector2d(Vector2d):
    typecode = 'f'

In [25]:
sv = ShortVector2d(1/11, 1/27)

In [26]:
sv

ShortVector2d(0.09090909090909091, 0.037037037037037035)

In [27]:
len(bytes(sv))

9

In [28]:
repr(sv)

'ShortVector2d(0.09090909090909091, 0.037037037037037035)'

# Vector

In [36]:
from array import array
from collections import abc
import reprlib
import math
import functools
import operator
import itertools


class Vector:
    __match_args__ = ('x', 'y', 'z', 't')

    typecode = 'd'

    def __init__(self, components) -> None:
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self) -> str:
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self) -> str:
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, __value: object) -> bool:
        # return tuple(self) == tuple(__value)
        # if len(self) != len(__value):
        #     return False
        # for a, b in zip(self, __value):
        #     if a != b:
        #         return False
        # return True
        if isinstance(__value, Vector):
            return len(self) == len(__value) and all(a == b for a, b in zip(self, __value))
        else:
            return NotImplemented

    def __hash__(self) -> int:  # 映射归约
        # hashes = (hash(x) for x in self._components)
        hashes = map(hash, self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)  # 返回新的对象
        except TypeError:
            return NotImplemented

    __radd__ = __add__  # 等效下面的函数

    # def __radd__(self, other):
    #     return self + other

    def __mul__(self, scalar):
        try:
            factor = float(scalar)
            return Vector(n*scalar for n in self)
        except TypeError:
            raise NotImplemented

    __rmul__ = __mul__

    def __matmul__(self, other):
        if (isinstance(other, abc.Sized) and
            isinstance(other, abc.Iterable)):
            if len(self) == len(other):
                return sum(a * b for a, b in zip(self, other))
            else:
                raise ValueError('@ requires vectors of equal length.')
        else:
            return NotImplemented

    __rmatmul__ = __matmul__

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    def __setattr__(self, __name: str, __value) -> None:
        cls = type(self)
        if len(__name) == 1:
            if __name in cls.__match_args__:
                error = 'readonly attribute {attr_name!r}'
            elif __name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=__name)
                raise AttributeError(msg)
        super().__setattr__(__name, __value)

    def angle(self, n):
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, __format_spec: str) -> str:
        if __format_spec.endswith('h'):
            __format_spec = __format_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, __format_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [37]:
v1 = Vector([3, 4, 5])
# v2 = Vector([])
v1 + (10, 20, 30)

Vector([13.0, 24.0, 35.0])

In [38]:
va = Vector([1, 2, 3])
vb = Vector(range(1,4))
va == vb

True

In [40]:
a = [1, 2, 3]
id(a)

1631123786560

In [41]:
a += [1,2,3]
id(a)

1631123786560

In [42]:
a

[1, 2, 3, 1, 2, 3]

In [39]:
va == [1,2,3]

False

In [33]:
v1 * 10

Vector([30.0, 40.0, 50.0])

In [35]:
[10, 20, 30] @ v1

260.0

In [34]:
v2 = Vector([2, 6, 7])
v1 @ v2

65.0

In [29]:
True * v1

Vector([3.0, 4.0, 5.0])

In [22]:
v1 + 1

TypeError: unsupported operand type(s) for +: 'Vector' and 'int'

In [16]:
(10, 20, 30) + v1

Vector([13.0, 24.0, 35.0])

In [76]:
import functools
import operator

functools.reduce(operator.xor, range(6))

1

In [80]:
list(zip(range(3), 'ABC', '123456'))

[(0, 'A', '1'), (1, 'B', '2'), (2, 'C', '3')]

In [81]:
from itertools import zip_longest

list(zip_longest(range(3), 'ABC', '123456'))


[(0, 'A', '1'),
 (1, 'B', '2'),
 (2, 'C', '3'),
 (None, None, '4'),
 (None, None, '5'),
 (None, None, '6')]

# 接口、协议和抽象基类

In [ ]:
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck():
    rankd

In [84]:
class Vowels:
    def __getitem__(self, i):
        return 'AEIOU'[i]

In [85]:
v = Vowels()
v[0]

'A'

In [87]:
v[2:5]

'IOU'

In [89]:
for c in v:
    print(c)

A
E
I
O
U


In [90]:
len(v)

TypeError: object of type 'Vowels' has no len()

In [91]:
'A' in v

True

In [92]:
from random import shuffle

l = list(range(10))

shuffle(l)
l

[3, 7, 0, 8, 5, 2, 9, 4, 6, 1]

In [93]:
class Struggle:
    def __len__(self):
        return 23

from collections import abc

isinstance(Struggle(), abc.Sized)

True

In [94]:
issubclass(Struggle, abc.Sized)

True

In [1]:
import abc

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """从可迭代对象中添加元素"""

    @abc.abstractmethod
    def pick(self):
        """随机删除元素，再返回被删除的元素。

        如果实例为空，那么这个方法应该抛出LookupError
        """

    def loaded(self):
        return bool(self.inspect())

    def inspect(self):
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(items)

In [2]:
import random

class BingoCage(Tombola):
    def __init__(self, items) -> None:
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        self.pick()

In [3]:
import random

class LottoBlower(Tombola):
    def __init__(self, iterable) -> None:
        self._balls = list(iterable)

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)

    def loaded(self):
        return bool(self._balls)

    def inspect(self):
        return tuple(self._balls)

In [4]:
from random import randrange

# @Tombola.register
class TomboList(list):
    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
    load = list.extend

    def loaded(self):
        return bool(self)

    def inspect(self):
        return tuple(self)

Tombola.register(TomboList)

__main__.TomboList

In [10]:
issubclass(TomboList, Tombola)

True

In [11]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [13]:
TomboList.__mro__

(__main__.TomboList, list, object)

# 静态协议

In [14]:
from typing import TypeVar, Protocol

T = TypeVar('T')

class Repeatable(Protocol):
    def __mul__(self: T, repeat_count: int) -> T: ...

RT = TypeVar('RT', bound=Repeatable)

def double(x: RT) -> RT:
    return x * 2

In [41]:
from typing import Protocol, runtime_checkable, Any, Iterable, TYPE_CHECKING

@runtime_checkable
class RandomPicker(Protocol):
    def pick(self) -> Any: ...

class SimplePicker:
    def __init__(self, items: Iterable) -> None:
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self) -> Any:
        return self._items.pop()

@runtime_checkable
class LoadableRandomPicker(RandomPicker, Protocol):  # 扩展协议
    def load(self, Iterable) -> None: ...

In [42]:
popper = SimplePicker([1])
isinstance(popper, RandomPicker)

True

In [43]:
popper.pick()

1

In [44]:
x = 1

isinstance(x, numbers.Integral)

True

In [45]:
import numpy as np

cd = np.cdouble(3+4j)
cd

(3+4j)

In [46]:
float(cd)

C:\Users\sanshi\AppData\Local\Temp\ipykernel_13548\3810435258.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  float(cd)


3.0

In [52]:
import collections

class DoppelDict(collections.UserDict):
    def __setitem__(self, __key: Any, __value: Any) -> None:
        super().__setitem__(__key, [__value] * 2)

In [53]:
dd = DoppelDict(one=1)
dd

{'one': [1, 1]}

In [54]:
dd['two'] = 2

In [55]:
dd

{'one': [1, 1], 'two': [2, 2]}

In [56]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [57]:
class Root:  # <1>
    def ping(self):
        print(f'{self}.ping() in Root')

    def pong(self):
        print(f'{self}.pong() in Root')

    def __repr__(self):
        cls_name = type(self).__name__
        return f'<instance of {cls_name}>'


class A(Root):  # <2>
    def ping(self):
        print(f'{self}.ping() in A')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in A')
        super().pong()


class B(Root):  # <3>
    def ping(self):
        print(f'{self}.ping() in B')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in B')


class Leaf(A, B):  # <4>
    def ping(self):
        print(f'{self}.ping() in Leaf')
        super().ping()

In [58]:
Leaf.__mro__

(__main__.Leaf, __main__.A, __main__.B, __main__.Root, object)

In [65]:
class U():
    def ping(self):
        print(f'{self}.ping() in U')
        super().ping()
class LeafUA(A, U):
    def ping(self):
        print(f'{self}.ping() in LeafUA')
        super().ping()

In [66]:
u = U()
ua = LeafUA()

In [61]:
u.ping()

<__main__.U object at 0x00000238E9DF30A0>.ping() in U


AttributeError: 'super' object has no attribute 'ping'

In [67]:
ua.ping()

<instance of LeafUA>.ping() in LeafUA
<instance of LeafUA>.ping() in A
<instance of LeafUA>.ping() in Root


In [68]:
LeafUA.__mro__  # A, U

(__main__.LeafUA, __main__.A, __main__.Root, __main__.U, object)

In [69]:
import collections

def _upper(key):
    try:
        return key.upper()
    except AttributeError:
        return key

class UpperCaseMixin:
    def __setitem__(self, key, item):
        super().__setitem__(_upper(key), item)

    def __getitem__(self, key):
        return super().__getitem__(_upper(key))

    def get(self, key, default=None):
        return super().get(_upper(key), default)

    def __contains__(self, key):
        return super().__contains__(_upper(key))


In [70]:
class UpperDict(UpperCaseMixin, collections.UserDict): ...

class UpperCounter(UpperCaseMixin, collections.Counter):
    """一个特殊的计数器"""

In [71]:
d = UpperDict([('a', 'letter A'), (2, 'digit two')])
d

{'A': 'letter A', 2: 'digit two'}

In [72]:
'a' in d

True

In [74]:
d['a'], d['A']

('letter A', 'letter A')

In [75]:
d.get('A')

'letter A'

In [76]:
list(d.keys())

['A', 2]

In [77]:
help(sum)

Help on built-in function sum in module builtins:

sum(iterable, /, start=0)
    Return the sum of a 'start' value (default: 0) plus an iterable of numbers
    
    When the iterable is empty, return the start value.
    This function is intended specifically for use with numeric values and may
    reject non-numeric types.



In [78]:
!python -V

Python 3.9.12


In [79]:
from typing import TypedDict

class BookDict(TypedDict):
    isbn: str
    title: str
    authors: list[str]
    pagecount: int

In [80]:
pp = BookDict(title='Programming Pearls',
              authors='Jon',
              isbn='0123',
              pagecount=256)

In [81]:
pp

{'title': 'Programming Pearls',
 'authors': 'Jon',
 'isbn': '0123',
 'pagecount': 256}

In [82]:
type(pp)

dict

In [83]:
pp.title

AttributeError: 'dict' object has no attribute 'title'

In [84]:
pp['title']  # type: ignore

'Programming Pearls'

In [88]:
type(BookDict.__annotations__['isbn'])

type

In [86]:
pp.__annotations__

AttributeError: 'dict' object has no attribute '__annotations__'

# 泛化类

In [7]:
import random

from collections.abc import Iterable
from typing import TypeVar, Generic

T = TypeVar('T')

class LottoBlower(Tombola, Generic[T]):

    def __init__(self, items: Iterable[T]) -> None:
        self._balls = list[T](items)

    def load(self, items: Iterable[T]) -> None:
        self._balls.extend(items)

    def pick(self) -> T:
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)

    def loaded(self) -> bool:
        return bool(self._balls)

    def inspect(self) -> tuple[T, ...]:
        return tuple(self._balls)

In [8]:
machine = LottoBlower[int](range(1, 11))
machine

# 型变

In [10]:
from typing import TypeVar, Generic

class Beverage:
    """任何饮料"""

class Juice(Beverage):
    """任何果汁"""

class OrangeJuice(Juice):
    """使用巴西橙子制作的美味果汁"""
# 不变
T = TypeVar('T')

class BeverageDispenser(Generic[T]):
    """参数化饮料类型的自动售货机"""
    def __init__(self, beverage: T) -> None:
        self.beverage = beverage

    def dispense(self) -> T:
        return self.beverage

def install(dispenser: BeverageDispenser[Juice]) -> None:
    """安装一个果汁自动售货机"""

In [11]:
install(Beverage())

In [12]:
# 协变
T_co = TypeVar('T_co', covariant=True)

class BeverageDispenser(Generic[T_co]):
    """参数化饮料类型的自动售货机"""
    def __init__(self, beverage: T_co) -> None:
        self.beverage = beverage

    def dispense(self) -> T_co:
        return self.beverage

def install(dispenser: BeverageDispenser[Juice]) -> None:
    """安装一个果汁自动售货机"""

In [ ]:
# 逆变
T_contra = TypeVar('T_contra', contravariant=True)

class BeverageDispenser(Generic[T_contra]):
    """参数化饮料类型的自动售货机"""
    def __init__(self, beverage: T_contra) -> None:
        self.beverage = beverage

    def dispense(self) -> T_contra:
        return self.beverage

def install(dispenser: BeverageDispenser[Juice]) -> None:
    """安装一个果汁自动售货机"""

In [1]:
# 泛化静态协议
...

# 运算符重载

In [2]:
import decimal
ctx = decimal.getcontext()
ctx.prec = 40
one_third = decimal.Decimal('1') / decimal.Decimal('3')
one_third

Decimal('0.3333333333333333333333333333333333333333')

In [4]:
one_third == +one_third

True

In [6]:
ctx.prec = 28
one_third == +one_third

False

In [7]:
+one_third

Decimal('0.3333333333333333333333333333')